In [ ]:
!wget http://www.manythings.org/anki/ita-eng.zip
!unzip ita-eng.zip

--2021-02-10 16:34:57--  http://www.manythings.org/anki/ita-eng.zip
Resolving www.manythings.org (www.manythings.org)... 172.67.173.198, 104.21.55.222, 2606:4700:3036::ac43:adc6, ...
Connecting to www.manythings.org (www.manythings.org)|172.67.173.198|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7567445 (7.2M) [application/zip]
Saving to: ‘ita-eng.zip’

ita-eng.zip         100%[===================>]   7.22M  6.21MB/s    in 1.2s    

2021-02-10 16:34:58 (6.21 MB/s) - ‘ita-eng.zip’ saved [7567445/7567445]

Archive:  ita-eng.zip
  inflating: ita.txt                 
  inflating: _about.txt              


In [ ]:
!wget https://www.dropbox.com/s/ddkmtqz01jc024u/glove.6B.100d.txt

--2021-02-10 16:35:04--  https://www.dropbox.com/s/ddkmtqz01jc024u/glove.6B.100d.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6027:18::a27d:4812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/ddkmtqz01jc024u/glove.6B.100d.txt [following]
--2021-02-10 16:35:04--  https://www.dropbox.com/s/raw/ddkmtqz01jc024u/glove.6B.100d.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc12ef80ec870547e21a5f9c12a6.dl.dropboxusercontent.com/cd/0/inline/BIqmOVLS1-PzgucETQAHNAI9SwUQILLdLg61XuNFH42CMgiWbVLShaJBzhSyNBfqpxZPLdY8dMlcscdPYzg5P3oBSnnENbPcxGnKawLRuCo1VaMb76LsDp8Ok6h_MQgxSPc/file# [following]
--2021-02-10 16:35:04--  https://uc12ef80ec870547e21a5f9c12a6.dl.dropboxusercontent.com/cd/0/inline/BIqmOVLS1-PzgucETQAHNAI9SwUQILLdLg61XuNFH42CMgiWbVLShaJBzhSyNBfqpxZPLdY8dMlcscdPYzg5P3oBSnnEN

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
# import seaborn as sns
import pandas as pd
import re
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [ ]:
with open('ita.txt', 'r', encoding="utf8") as f:
    eng=[]
    ita=[]
    for i in f.readlines():
        eng.append(i.split("\t")[0])
        ita.append(i.split("\t")[1])
data = pd.DataFrame(data=list(zip(eng, ita)), columns=['english','italian'])
print(data.shape)
data.head()

(345244, 2)


,english,italian
0,Hi.,Ciao!
1,Run!,Corri!
2,Run!,Corra!
3,Run!,Correte!
4,Who?,Chi?


In [ ]:
def decontractions(phrase):
    """decontracted takes text and convert contractions into natural form.
     ref: https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python/47091490#47091490"""
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"won\’t", "will not", phrase)
    phrase = re.sub(r"can\’t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)

    phrase = re.sub(r"n\’t", " not", phrase)
    phrase = re.sub(r"\’re", " are", phrase)
    phrase = re.sub(r"\’s", " is", phrase)
    phrase = re.sub(r"\’d", " would", phrase)
    phrase = re.sub(r"\’ll", " will", phrase)
    phrase = re.sub(r"\’t", " not", phrase)
    phrase = re.sub(r"\’ve", " have", phrase)
    phrase = re.sub(r"\’m", " am", phrase)

    return phrase

def preprocess(text):
    # convert all the text into lower letters
    # use this function to remove the contractions: https://gist.github.com/anandborad/d410a49a493b56dace4f814ab5325bbd
    # remove all the spacial characters: except space ' '
    text = text.lower()
    text = decontractions(text)
    text = re.sub('[^A-Za-z0-9 ]+', '', text)
    return text

def preprocess_ita(text):
    # convert all the text into lower letters
    # remove the words betweent brakets ()
    # remove these characters: {'$', ')', '?', '"', '’', '.',  '°', '!', ';', '/', "'", '€', '%', ':', ',', '('}
    # replace these spl characters with space: '\u200b', '\xa0', '-', '/'
    # we have found these characters after observing the data points, feel free to explore more and see if you can do find more
    # you are free to do more proprocessing
    # note that the model will learn better with better preprocessed data

    text = text.lower()
    text = decontractions(text)
    text = re.sub('[$)\?"’.°!;\'€%:,(/]', '', text)
    text = re.sub('\u200b', ' ', text)
    text = re.sub('\xa0', ' ', text)
    text = re.sub('-', ' ', text)
    return text


data['english'] = data['english'].apply(preprocess)
data['italian'] = data['italian'].apply(preprocess_ita)
data.head()

,english,italian
0,hi,ciao
1,run,corri
2,run,corra
3,run,correte
4,who,chi


In [ ]:
ita_lengths = data['italian'].str.split().apply(len)
eng_lengths = data['english'].str.split().apply(len)

In [ ]:
data['italian_len'] = data['italian'].str.split().apply(len)
data = data[data['italian_len'] < 20]

data['english_len'] = data['english'].str.split().apply(len)
data = data[data['english_len'] < 20]

data['italian_inp'] = '<start> ' + data['italian'].astype(str)
data['italian_out'] = data['italian'].astype(str) + ' <end>'

data = data.drop(['italian','italian_len','english_len'], axis=1)
# only for the first sentance add a toke <end> so that we will have <end> in tokenizer
data.head()

,english,italian_inp,italian_out
0,hi,<start> ciao,ciao <end>
1,run,<start> corri,corri <end>
2,run,<start> corra,corra <end>
3,run,<start> correte,correte <end>
4,who,<start> chi,chi <end>


In [ ]:
from sklearn.model_selection import train_test_split
train, validation = train_test_split(data, test_size=0.2)

In [ ]:
print(train.shape, validation.shape)
# for one sentence we will be adding <end> token so that the tokanizer learns the word <end>
# with this we can use only one tokenizer for both encoder output and decoder output
train.iloc[0]['italian_inp']= str(train.iloc[0]['italian_inp'])+' <end>'
train.iloc[0]['italian_out']= str(train.iloc[0]['italian_out'])+' <end>'

(275852, 3) (68964, 3)


In [ ]:
train.head()

,english,italian_inp,italian_out
209416,i cut my finger with a knife,<start> io mi sono tagliato il dito con un col...,io mi sono tagliato il dito con un coltello <e...
97108,why were you so rude,<start> perché eri così maleducato,perché eri così maleducato <end>
63349,she is my godmother,<start> è la mia madrina,è la mia madrina <end>
130679,have you registered yet,<start> lei si è già registrato,lei si è già registrato <end>
64596,tom began shouting,<start> tom iniziò a gridare,tom iniziò a gridare <end>


In [ ]:
tknizer_ita = Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')
tknizer_ita.fit_on_texts(train['italian_inp'].values)
tknizer_eng = Tokenizer()
tknizer_eng.fit_on_texts(train['english'].values)

In [ ]:
vocab_size_eng=len(tknizer_eng.word_index.keys())
print(vocab_size_eng)
vocab_size_ita=len(tknizer_ita.word_index.keys())
print(vocab_size_ita)

12863
26237


In [ ]:
tknizer_ita.word_index['<start>'], tknizer_ita.word_index['<end>']

(1, 18773)

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
class Encoder(tf.keras.Model):
    '''
    Encoder model -- That takes a input sequence and returns encoder-outputs,encoder_final_state_h,encoder_final_state_c
    '''

    def __init__(self,inp_vocab_size,embedding_size,lstm_size,input_length):

        #Initialize Embedding layer
        #Intialize Encoder LSTM layer
        super().__init__()
        self.vocab_size = inp_vocab_size
        self.embedding_dim = embedding_size
        self.input_length = input_length
        self.lstm_size= lstm_size
        self.lstm_output = 0
        self.lstm_state_h=0
        self.lstm_state_c=0
        #self.states = 0
    def build(self,input_shape):
        self.states = tf.zeros((1024,self.lstm_size)) , tf.zeros((1024,self.lstm_size))
        self.embedding = Embedding(input_dim=self.vocab_size, output_dim=self.embedding_dim, input_length=self.input_length,
                           mask_zero=True, name="embedding_layer_encoder")
        self.lstm = LSTM(self.lstm_size, return_state=True, return_sequences=True, name="Encoder_LSTM")


    def call(self,input_sequence,states):
        '''
          This function takes a sequence input and the initial states of the encoder.
          Pass the input_sequence input to the Embedding layer, Pass the embedding layer ouput to encoder_lstm
          returns -- encoder_output, last time step's hidden and cell state
        '''
        #print("ENCODER ==> INPUT SQUENCES SHAPE :",input_sequence.shape)
        input_embedd                           = self.embedding(input_sequence)
        #print("ENCODER ==> AFTER EMBEDDING THE INPUT SHAPE :",input_embedd.shape)
        self.lstm_output, self.lstm_state_h,self.lstm_state_c = self.lstm(input_embedd)
        #print(self.lstm_output.shape)
        return self.lstm_output, self.lstm_state_h,self.lstm_state_c




    def initialize_states(self,batch_size):
      '''
      Given a batch size it will return intial hidden state and intial cell state.
      If batch size is 32- Hidden state is zeros of size [32,lstm_units], cell state zeros is of size [32,lstm_units]
      '''
      return tf.zeros((batch_size,self.lstm_size)) , tf.zeros((batch_size,self.lstm_size))
    def get_states(self):
      return self.lstm_state_h,self.lstm_state_c

In [ ]:
class Decoder(tf.keras.Model):
    '''
    Encoder model -- That takes a input sequence and returns output sequence
    '''

    def __init__(self,out_vocab_size,embedding_size,lstm_size,input_length):

        #Initialize Embedding layer
        #Intialize Decoder LSTM layer
        super().__init__()
        self.out_vocab_size = out_vocab_size
        self.embedding_size = embedding_size
        self.lstm_size = lstm_size
        self.input_length = input_length
        self.embedding = Embedding(input_dim=self.out_vocab_size, output_dim=self.embedding_size, input_length=self.input_length,
                           mask_zero=True, name="embedding_layer_decoder")
        self.lstm = LSTM(self.lstm_size, return_sequences=True, return_state=True, name="Decoder_LSTM")

    def call(self,input_sequence,initial_states):
        '''
          This function takes a sequence input and the initial states of the encoder.
          Pass the input_sequence input to the Embedding layer, Pass the embedding layer ouput to decoder_lstm

          returns -- decoder_output,decoder_final_state_h,decoder_final_state_c
        '''
        #print("DECODER ==> INPUT SQUENCES SHAPE :",input_sequence.shape)
        target_embedd           = self.embedding(input_sequence)
        #print("WE ARE INITIALIZING DECODER WITH ENCODER STATES :",initial_states[0].shape,initial_states[1].shape)
        lstm_output, x,y        = self.lstm(target_embedd, initial_state=initial_states)
        #print(lstm_output.shape)
        return lstm_output,x,y

In [ ]:
class Dataset:
    def __init__(self, data, tknizer_ita, tknizer_eng, max_len):
        self.encoder_inps = data['english'].values
        self.decoder_inps = data['italian_inp'].values
        self.decoder_outs = data['italian_out'].values
        self.tknizer_eng = tknizer_eng
        self.tknizer_ita = tknizer_ita
        self.max_len = max_len

    def __getitem__(self, i):
        self.encoder_seq = tknizer_eng.texts_to_sequences([self.encoder_inps[i]]) # need to pass list of values
        self.decoder_inp_seq = tknizer_ita.texts_to_sequences([self.decoder_inps[i]])
        self.decoder_out_seq = tknizer_ita.texts_to_sequences([self.decoder_outs[i]])

        self.encoder_seq = pad_sequences(self.encoder_seq, maxlen=self.max_len, dtype='int32', padding='post')
        self.decoder_inp_seq = pad_sequences(self.decoder_inp_seq, maxlen=self.max_len, dtype='int32', padding='post')
        self.decoder_out_seq = pad_sequences(self.decoder_out_seq, maxlen=self.max_len, dtype='int32', padding='post')
        return self.encoder_seq, self.decoder_inp_seq, self.decoder_out_seq

    def __len__(self): # your model.fit_gen requires this function
        return len(self.encoder_inps)


class Dataloder(tf.keras.utils.Sequence):
    def __init__(self, dataset, batch_size=1):
        self.dataset = dataset
        self.batch_size = batch_size
        self.indexes = np.arange(len(self.dataset.encoder_inps))


    def __getitem__(self, i):
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])

        batch = [np.squeeze(np.stack(samples, axis=1), axis=0) for samples in zip(*data)]
        # we are creating data like ([italian, english_inp], english_out) these are already converted into seq
        return tuple([[batch[0],batch[1]],batch[2]])

    def __len__(self):  # your model.fit_gen requires this function
        return len(self.indexes) // self.batch_size

    def on_epoch_end(self):
        self.indexes = np.random.permutation(self.indexes)

In [ ]:
class Encoder_decoder(tf.keras.Model):

    def __init__(self,*params):

        #Create encoder object
        #Create decoder object
        #Intialize Dense layer(out_vocab_size) with activation='softmax'
        super().__init__() # https://stackoverflow.com/a/27134600/4084039
        self.encoder = Encoder(inp_vocab_size=vocab_size_ita+1, embedding_size=50, input_length=encoder_inputs_length, lstm_size=256)
        self.decoder = Decoder(out_vocab_size=vocab_size_eng+1, embedding_size=50, input_length=decoder_inputs_length, lstm_size=256)
        self.dense   = Dense(output_vocab_size, activation='softmax')


    def call(self,data,batch_size=1024):
        '''
        A. Pass the input sequence to Encoder layer -- Return encoder_output,encoder_final_state_h,encoder_final_state_c
        B. Pass the target sequence to Decoder layer with intial states as encoder_final_state_h,encoder_final_state_C
        C. Pass the decoder_outputs into Dense layer


        Return decoder_outputs
        '''
        input,output =data[0], data[1]
        #print(data)
        #print("===========")
        #print(data[0])
        #print("+==")
        #print(data[0][1])
        #print("===========")
        #print(input.shape)
        #print('output')
        #print(type(output))
        #encoder_output, encoder_h, encoder_c = self.encoder(input)
        states = self.encoder.initialize_states(batch_size)
        encoder_output,encoder_final_state_h,encoder_final_state_c = self.encoder(input,states)
        decoder_output,_,_ = self.decoder(output,[encoder_final_state_h,encoder_final_state_c])
        output = self.dense(decoder_output)
        return output


In [ ]:
train_dataset = Dataset(train, tknizer_ita, tknizer_eng, 20)
test_dataset  = Dataset(validation, tknizer_ita, tknizer_eng, 20)


train_dataloader = Dataloder(train_dataset, batch_size=1024)
test_dataloader = Dataloder(test_dataset, batch_size=1024)


print(train_dataloader[0][0][0].shape, train_dataloader[0][0][1].shape, train_dataloader[0][1].shape)
print(test_dataloader[0][0][0].shape, test_dataloader[0][0][1].shape, test_dataloader[0][1].shape)

(1024, 20) (1024, 20) (1024, 20)
(1024, 20) (1024, 20) (1024, 20)


In [ ]:



#Create an object of encoder_decoder Model class,
# Compile the model and fit the model
encoder_inputs_length = 20
decoder_inputs_length = 20
lstm_size=64
output_vocab_size = vocab_size_eng
encoder_decoder = Encoder_decoder(vocab_size_eng, vocab_size_ita,encoder_inputs_length,decoder_inputs_length,output_vocab_size)
optimizer = tf.keras.optimizers.Adam()
encoder_decoder.compile(optimizer=optimizer,loss='sparse_categorical_crossentropy')


In [ ]:
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint
tensorboard_callback = TensorBoard(log_dir='logs', histogram_freq=1, write_graph=True)
model_callback = ModelCheckpoint('model.hdf5',monitor='val_loss',save_best_only=True,mode='min')

In [ ]:
!rm -rf logs/

In [ ]:
train_steps=train.shape[0]//1024
valid_steps=validation.shape[0]//1024
batch_size = 1024
encoder_decoder.fit(train_dataloader, steps_per_epoch=train_steps, epochs=50, validation_data=train_dataloader,validation_steps=valid_steps,callbacks=[tensorboard_callback,model_callback])
encoder_decoder.summary()

InvalidArgumentError: ignored

In [ ]:
encoder_decoder.fit(train_dataloader, steps_per_epoch=train_steps, epochs=30, validation_data=train_dataloader,validation_steps=valid_steps,callbacks=[tensorboard_callback,model_callback])

Epoch 1/30
269/269 [==============================] - 44s 165ms/step - loss: 0.0901 - val_loss: 0.0887
Epoch 2/30
269/269 [==============================] - 44s 164ms/step - loss: 0.0898 - val_loss: 0.0884
Epoch 3/30
269/269 [==============================] - 45s 165ms/step - loss: 0.0895 - val_loss: 0.0882
Epoch 4/30
269/269 [==============================] - 44s 165ms/step - loss: 0.0892 - val_loss: 0.0878
Epoch 5/30
269/269 [==============================] - 44s 164ms/step - loss: 0.0889 - val_loss: 0.0876
Epoch 6/30
269/269 [==============================] - 44s 163ms/step - loss: 0.0886 - val_loss: 0.0873
Epoch 7/30
269/269 [==============================] - 44s 164ms/step - loss: 0.0883 - val_loss: 0.0871
Epoch 8/30
269/269 [==============================] - 44s 162ms/step - loss: 0.0881 - val_loss: 0.0867
Epoch 9/30
269/269 [==============================] - 44s 163ms/step - loss: 0.0878 - val_loss: 0.0866
Epoch 10/30
269/269 [==============================] - 44s 165ms/step - l

In [ ]:
encoder_decoder.load_weights('model.hdf5')

In [ ]:
def predict(input_sentence):
  '''
  A. Given input sentence, convert the sentence into integers using tokenizer used earlier
  B. Pass the input_sequence to encoder. we get encoder_outputs, last time step hidden and cell state
  C. Initialize index of <start> as input to decoder. and encoder final states as input_states to decoder
  D. till we reach max_length of decoder or till the model predicted word <end>:
         predicted_out,state_h,state_c=model.layers[1](dec_input,states)
         pass the predicted_out to the dense layer
         update the states=[state_h,state_c]
         And get the index of the word with maximum probability of the dense layer output, using the tokenizer(word index) get the word and then store it in a string.
         Update the input_to_decoder with current predictions
  F. Return the predicted sentence
  '''
  prev_word = ""
  string = ""
  test_sent = input_sentence
  test_sent_seq = tknizer_eng.texts_to_sequences([test_sent])
  test_sent_seq = pad_sequences(test_sent_seq, maxlen=20, dtype='int32', padding='post')
  states = encoder_decoder.encoder.get_states()
  #print(states)
  lstm_output,state_h,state_c = encoder_decoder.encoder(tf.convert_to_tensor(test_sent_seq),states)
  #(1, 10119)
  tknizer_ita.texts_to_sequences(['<start>'])
  #decoder_out,state_dec_h,state_dec_c = encoder_decoder.decoder(tknizer_eng.texts_to_sequences(['<start>']),[state_h,state_c])
  inp = tknizer_ita.texts_to_sequences(['<start>'])
  inp = tf.convert_to_tensor(inp)
  for i in range(0,20):
    #print(i)
    #print(inp)
    decoder_out,state_dec_h,state_dec_c = encoder_decoder.decoder(inp,[state_h,state_c])
    d = encoder_decoder.dense(decoder_out)
    state_h = state_dec_h
    state_c = state_dec_c
    #print(d[0][0])
    #print(ind)
    #print()
    ind = int(tf.math.argmax(d[0][0]))
    word = str(tknizer_ita.sequences_to_texts([[ind]])[0])

    if word == prev_word:
      break
    prev_word = word
    if(word=="<end>"):
      break
    string += str(tknizer_ita.sequences_to_texts([[ind]])[0])+" "
    #print(word)
    #print(decoder_out)
    inp = tf.convert_to_tensor(tknizer_ita.texts_to_sequences([word]))

  return string[:-1]

In [ ]:
list_of_sentences = validation['english'].sample(10)
for sentence in list_of_sentences:
  print(sentence)
  print(predict(sentence))
  print("====================================================")

she speaks chinese
compito la con ferito
i broke it
orsetti posò informatico ferito
i was upset
ho cominci lonestà posò ferito
who gave you these
non posò ferito
tom wrote love letters to mary
compito nellesercito modesto la ferito
we can not go on like this
riscaldando informatica <start> ferito
why are you crying tom
non posò non ferito
i am jittery
compito suo compito la ferito
did you do your homework by yourself
era
he promised me that he would come at four
compito posò aiutarmi riscaldando la cugini ferito
